In [148]:
# Imports
import pandas as pd
import numpy as np
import json as json
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [149]:
 # Loading the data from CSV into panadas dataframe
tempIncomeStatement = pd.read_csv (r'../data/income_statement.csv', sep=',')
tempCashFlow = pd.read_csv (r'../data/cash_flow.csv', sep=',')
tempBalanceSheet = pd.read_csv (r'../data/balance-sheet.csv', sep=',')
tempEarnings = pd.read_csv (r'../data/EARNINGS.csv', sep=',')

tempMonthly = pd.read_csv (r'../data/monthly-data.csv', sep=',')
TickerSymbols = pd.read_csv (r'../data/nasdaq_screener_sorted_values.csv', sep=',')

In [150]:
# Printing Income statement
print(tempIncomeStatement)
print(tempIncomeStatement.columns)

# Printing Cash Flow
print(tempCashFlow.shape)
print(tempCashFlow.columns)

# Printing Balance Sheet
print(tempBalanceSheet.shape)
print(tempBalanceSheet.columns)

# Printing Earnings
print(tempEarnings.shape)
print(tempEarnings.columns)

     fiscalDateEnding reportedCurrency  grossProfit  totalRevenue  \
0          2021-09-30              USD  35174000000   82688000000   
1          2021-06-30              USD  35255000000   80769000000   
2          2021-03-31              USD  38079000000   88914000000   
3          2020-12-31              USD  44328000000  110801000000   
4          2020-09-30              USD  24689000000   64698000000   
...               ...              ...          ...           ...   
3899       2017-05-31              USD    345500000     572700000   
3900       2017-02-28              USD    325900000     566400000   
3901       2016-11-30              USD    356900000     623700000   
3902       2016-08-31              USD    326900000     597300000   
3903       2016-05-31              USD    248800000     710500000   

     costOfRevenue costofGoodsAndServicesSold operatingIncome  \
0      53802000000                48186000000     23786000000   
1      51591000000                4617900

In [151]:
# renaming Balance sheet column "fiscalDateEnding" to "timestamp" to match column name on both dataframes
tempIncomeStatement = tempIncomeStatement.rename(columns={"fiscalDateEnding": "timestamp"})
tempCashFlow = tempCashFlow.rename(columns={"fiscalDateEnding": "timestamp"})
tempBalanceSheet = tempBalanceSheet.rename(columns={"fiscalDateEnding": "timestamp"})
tempEarnings = tempEarnings.rename(columns={"fiscalDateEnding": "timestamp"})

# Merging all 4 frames into one

# Daily dates does not overlap on both dataframes, CompanyMonthly has the last trading day (not including weekends)
# While CompanyBalanaceSheet always has the last day of the month (including weekends)
# We need to cut out the daily date for both dataframes["timestamp"], to be able to merge them properly.
# Slicing the timestamp in CompanyMonthly, removing the days
sliceMonthly = tempMonthly["timestamp"].str.slice(0, -3)
tempMonthly["timestamp"] = sliceMonthly

# Printing
print("Monthly Data:")
print(tempMonthly["timestamp"])

# Slicing the timestamp in Income Statement, removing the days
sliceIncomestatement = tempIncomeStatement["timestamp"].str.slice(0, -3)
# For some reason i needed to use Iloc here, don't know why... 
tempIncomeStatement["timestamp"] = sliceIncomestatement

# Slicing the timestamp in Cash Flow, removing the days
sliceCashFlow = tempCashFlow["timestamp"].str.slice(0, -3)
# For some reason i needed to use Iloc here, don't know why... 
tempCashFlow["timestamp"] = sliceCashFlow

# Slicing the timestamp in CompanyBalanceSheet, removing the days
sliceBalanceSheet = tempBalanceSheet["timestamp"].str.slice(0, -3)
# For some reason i needed to use Iloc here, don't know why... 
tempBalanceSheet["timestamp"] = sliceBalanceSheet

# Slicing the timestamp in Earnings, removing the days
sliceEarnings = tempEarnings["timestamp"].str.slice(0, -3)
# For some reason i needed to use Iloc here, don't know why... 
tempEarnings["timestamp"] = sliceEarnings

# Printing
print("Income Statement:")
print(tempIncomeStatement["timestamp"])
print("Cash Flow:")
print(tempCashFlow["timestamp"])
print("Balance Sheet:")
print(tempBalanceSheet["timestamp"])
print("Earnings:")
print(tempEarnings["timestamp"])
print("Ticker Symbols:")
print(TickerSymbols["Symbol"])
    

Monthly Data:
0        2021-11
1        2021-10
2        2021-09
3        2021-08
4        2021-07
          ...   
43662    2000-11
43663    2000-10
43664    2000-09
43665    2000-08
43666    2000-07
Name: timestamp, Length: 43667, dtype: object
Income Statement:
0       2021-09
1       2021-06
2       2021-03
3       2020-12
4       2020-09
         ...   
3899    2017-05
3900    2017-02
3901    2016-11
3902    2016-08
3903    2016-05
Name: timestamp, Length: 3904, dtype: object
Cash Flow:
0       2021-09
1       2021-06
2       2021-03
3       2020-12
4       2020-09
         ...   
3880    2017-05
3881    2017-02
3882    2016-11
3883    2016-08
3884    2016-05
Name: timestamp, Length: 3885, dtype: object
Balance Sheet:
0       2021-09
1       2021-06
2       2021-03
3       2020-12
4       2020-09
         ...   
3852    2017-05
3853    2017-02
3854    2016-11
3855    2016-08
3856    2016-05
Name: timestamp, Length: 3857, dtype: object
Earnings:
0        2021-09
1        2021-06
2 

In [152]:
# for each company we want to make adjustments needed for each companys features and then we will merge them into one df

#------------ add error if ticker symbol is not in TickerSymbols["Symbol"] or size < 200


TickerSymbols = TickerSymbols[~TickerSymbols['Symbol'].isin(['RIVN'])]
final_df = pd.DataFrame()
counter = 0

for tickerSymbol in TickerSymbols["Symbol"]:
    counter = counter +1
    #print("this is the beginning of the forloop, tickersymbol: " + tickerSymbol + "counter = " + str(counter))
    
    # filter all dataframes for each ticketsymbol (stock)
    CompanyIncomeStatement = tempIncomeStatement.loc[tempIncomeStatement["symbol"] == tickerSymbol]
    CompanyCashFlow = tempCashFlow.loc[tempCashFlow["symbol"] == tickerSymbol]
    CompanyBalanceSheet = tempBalanceSheet.loc[tempBalanceSheet["symbol"] == tickerSymbol]
    CompanyEarnings = tempEarnings.loc[tempEarnings["symbol"] == tickerSymbol]
    CompanyMonthly = tempMonthly.loc[tempMonthly["Symbol"] == tickerSymbol]
    
    # Removing symbol for all Company dataframes
    CompanyIncomeStatement.drop(["reportedCurrency", 'symbol'], axis=1, inplace=True)
    CompanyCashFlow.drop(["reportedCurrency", 'symbol'], axis=1, inplace=True)
    CompanyBalanceSheet.drop(["reportedCurrency", 'symbol'], axis=1, inplace=True)
    CompanyEarnings.drop(['symbol', 'reportedDate'], axis=1, inplace=True)
    CompanyMonthly.drop(['Symbol'], axis=1, inplace=True)
    
    # Cutting out unecessary data from the CompanyMonthly
    # We only need price data in CompanyMonthly within the daterange of the 20 rows on CompanyBalanceSheet data
    # every row is 3 months in CompanyBalanceSheet. Every row in CompanyMonthly is 1 month, 3x20 = 60.
    idx = (CompanyMonthly.index[0]+60)
    CompanyMonthly = CompanyMonthly.loc[:idx]

    # calculating 1 month labels
    CompanyMonthly["1m"] = CompanyMonthly["close"].pct_change(periods = -1).shift(periods = 1)
    #print(CompanyMonthly.head(21))
    
    # merge all df into a temp df
    temp_df = pd.merge(CompanyIncomeStatement, CompanyCashFlow, how="outer", on = "timestamp")
    temp_df = pd.merge(temp_df, CompanyBalanceSheet, how="outer", on = "timestamp")
    temp_df = pd.merge(temp_df, CompanyEarnings, how="outer", on = "timestamp")
    #print(temp_df)
    
    # merging the rows from balance sheet and monthly on timestamp.
    merged_df = pd.merge(temp_df, CompanyMonthly, how="outer", on = "timestamp")
    
    # removing the rows over 20
    merged_df = merged_df.loc[:19]

    # calculate how much the increase is in percentage, interval of 3 months
    # also offsetting by 1, to get the labels on the correct row.
    merged_df["3m"] = merged_df["close"].pct_change(periods = -1).shift(periods = 1)
    
    # Removing the first row, because we do not have a label for it, 3 months have not passed yet for us to get the end price of the 3 month cycle
    merged_df.drop(index=merged_df.index[0], axis=0, inplace=True)
    merged_df.drop(index=merged_df.index[0], axis=0, inplace=True)

    # Also dropping columns that we do not need, symbol, Symbol, open, high, low, close, volume etc.
    merged_df.drop(['open','high','low','volume', "close"], axis=1, inplace=True)

    # replacing all "None" strings with NaN data type
    merged_df = merged_df.replace('None', np.nan)
    
    for col in merged_df.columns:
        if col != "timestamp":
            if col != "1m":
                if col != "3m":
                    merged_df[col] = merged_df[col].astype("float")
                    merged_df[col] = merged_df[col].replace(np.nan, merged_df[col].mean()) 
                    merged_df[col] = merged_df[col].pct_change(periods = -1).shift(periods = 0)

    # adding it to the final dataframe
    #print(str(col) + " Percentage difference: ")
    #print(merged_df.head(10))
    
    
    final_df = pd.concat([final_df, merged_df], axis=0)
    
print(final_df.head(25))
sub_df = final_df.loc[merged_df["1m"] > 1]
print(sub_df)

C:\Users\Asus\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


   timestamp  grossProfit  totalRevenue  costOfRevenue  \
1    2021-06    -0.074162     -0.091605      -0.092011   
2    2021-03    -0.140972     -0.197534      -0.218897   
3    2020-12     0.795455      0.712588       0.618467   
4    2020-09     0.088580      0.096799       0.074314   
5    2020-06     0.013858      0.024880       0.023010   
6    2020-03    -0.364795     -0.367753      -0.338258   
7    2019-12     0.448484      0.421518       0.394854   
8    2019-09     0.202007      0.209603       0.165676   
9    2019-06    -0.073049     -0.071257      -0.065040   
10   2019-03    -0.318754     -0.316651      -0.287582   
11   2018-12     0.329970      0.326232       0.470064   
12   2018-09     0.179374      0.180888       0.181829   
13   2018-06    -0.128127     -0.128760      -0.129153   
14   2018-03    -0.309330     -0.307567      -0.306467   
15   2017-12     0.701470      0.679245       0.665676   
16   2017-09     0.139696      0.157924       0.169341   
17   2017-06  

In [153]:
print(final_df)
print(final_df.columns)
print(final_df.dtypes)
cat = ["1m", "3m"]
print(final_df[cat].head(20))


   timestamp  grossProfit  totalRevenue  costOfRevenue  \
1    2021-06    -0.074162     -0.091605      -0.092011   
2    2021-03    -0.140972     -0.197534      -0.218897   
3    2020-12     0.795455      0.712588       0.618467   
4    2020-09     0.088580      0.096799       0.074314   
5    2020-06     0.013858      0.024880       0.023010   
..       ...          ...           ...            ...   
15   2017-08     0.057019      0.056050       0.054577   
16   2017-05     0.060141      0.011123      -0.054908   
17   2017-02    -0.086859     -0.091871      -0.098951   
18   2016-11     0.091771      0.044199      -0.013314   
19   2016-08          NaN           NaN            NaN   

    costofGoodsAndServicesSold  operatingIncome  \
1                    -0.103407        -0.122787   
2                    -0.232540        -0.179847   
3                     0.677398         1.269645   
4                     0.081178         0.128638   
5                     0.029547         0.018517 

In [154]:
print(final_df["1m"].isna().sum())
print(final_df["3m"].isna().sum())

136
0


In [155]:
# deleting columns with more than 1 % nan values
m = len(final_df)
for col in merged_df.columns:
    if col != "1m" or col != "3m":
        percentageNan = ((final_df[col].isnull().sum()/m)*100)
        print(str(col) + " has " + str(percentageNan))

        if ((final_df[col].isnull().sum()/m)*100) >= 0.1:
            final_df.drop(col, axis=1, inplace=True)
            print(str(col) + " has been dropped.")


timestamp has 0.0
grossProfit has 5.713526441668881
grossProfit has been dropped.
totalRevenue has 5.421206484188148
totalRevenue has been dropped.
costOfRevenue has 6.351315439808664
costOfRevenue has been dropped.
costofGoodsAndServicesSold has 6.989104437948446
costofGoodsAndServicesSold has been dropped.
operatingIncome has 5.607228275312251
operatingIncome has been dropped.
sellingGeneralAndAdministrative has 5.554079192133936
sellingGeneralAndAdministrative has been dropped.
researchAndDevelopment has 40.473026840287005
researchAndDevelopment has been dropped.
operatingExpenses has 5.474355567366463
operatingExpenses has been dropped.
investmentIncomeNet has 69.12038267339888
investmentIncomeNet has been dropped.
netInterestIncome has 9.593409513685888
netInterestIncome has been dropped.
interestIncome has 8.078660643103905
interestIncome has been dropped.
interestExpense has 8.238107892638851
interestExpense has been dropped.
nonInterestIncome has 32.36779165559394
nonInterestIn

In [156]:
final_df.isnull().sum()
final_df.shape

(3763, 2)

In [157]:
# correlation calculation
feature_corr = final_df.copy().corr()

# sorting and using 1m
sortedDesc = feature_corr["1m"].sort_values(ascending=False) 
print(sortedDesc.head(50))

#cf_matrix = confusion_matrix(feature_corr)
plt.figure(figsize = (24,18))
sns.heatmap(feature_corr, annot=True, cbar=False, )

KeyError: '1m'

In [ ]:
# sorting and using 3m
sortedDesc = feature_corr["3m"].sort_values(ascending=False) 
print(sortedDesc.head(50))

#cf_matrix = confusion_matrix(feature_corr)
plt.figure(figsize = (48,24))
sns.heatmap(feature_corr, annot=True, cbar=False, )